In [32]:
%env DATA_BUCKET=tbd-2023z-304098-data
%env GEN_OUTPUT_DIR=/tmp/tpc-di
%env REPO_ROOT=/home/jupyter/git/tbd-tpc-di/

env: DATA_BUCKET=tbd-2023z-304098-data
env: GEN_OUTPUT_DIR=/tmp/tpc-di
env: REPO_ROOT=/home/jupyter/git/tbd-tpc-di/


In [33]:
!pip3.8 install typer[All]==0.9.0 google-cloud-storage==2.13.0

## Install SDKMAN for setting up JVM 8 enviroment

In [34]:
%%bash
curl -s https://get.sdkman.io | bash


                                -+syyyyyyys:
                            `/yho:`       -yd.
                         `/yh/`             +m.
                       .oho.                 hy                          .`
                     .sh/`                   :N`                `-/o`  `+dyyo:.
                   .yh:`                     `M-          `-/osysoym  :hs` `-+sys:      hhyssssssssy+
                 .sh:`                       `N:          ms/-``  yy.yh-      -hy.    `.N-````````+N.
               `od/`                         `N-       -/oM-      ddd+`     `sd:     hNNm        -N:
              :do`                           .M.       dMMM-     `ms.      /d+`     `NMMs       `do
            .yy-                             :N`    ```mMMM.      -      -hy.       /MMM:       yh
          `+d+`           `:/oo/`       `-/osyh/ossssssdNMM`           .sh:         yMMN`      /m.
         -dh-           :ymNMMMMy  `-/shmNm-`:N/-.``   `.sN            /N-         `NMMy      .m/
  

## Install and set as default JVM 8

In [35]:
%%bash
source "$HOME/.sdkman/bin/sdkman-init.sh"
sdk install java 8.0.392-amzn
sdk use java 8.0.392-amzn


java 8.0.392-amzn is already installed.

Using java version 8.0.392-amzn in this shell.


## Check if JVM 8 is used

In [36]:
%%bash
source "$HOME/.sdkman/bin/sdkman-init.sh"
java -version

openjdk version "1.8.0_392"
OpenJDK Runtime Environment Corretto-8.392.08.1 (build 1.8.0_392-b08)
OpenJDK 64-Bit Server VM Corretto-8.392.08.1 (build 25.392-b08, mixed mode)


## Clone tbd-tpc-di repo

In [37]:
%%bash
mkdir -p git && cd git
# git clone https://github.com/thai-chicken/tbd-tpc-di.git
cd tbd-tpc-di
# git pull
git checkout notebook

Already on 'notebook'


Your branch is up to date with 'origin/notebook'.


## Generate input dataset (run this cell below from the terminal!!!)
It should take approx. 15min with scale factor set to 100 and generate approx. 10GiB

In [ ]:
%%script bash
source "$HOME/.sdkman/bin/sdkman-init.sh"
cd /home/jupyter/git/tbd-tpc-di/tools/ 
java -jar DIGen.jar -sf 10 -o /tmp/tpc-di

In [ ]:
# Install and setup JVM 11

In [26]:
%%bash
source "$HOME/.sdkman/bin/sdkman-init.sh"
sdk install java 11.0.21-amzn
sdk use java 11.0.21-amzn -y


java 11.0.21-amzn is already installed.

Using java version 11.0.21-amzn in this shell.


## Load staging

In [38]:
%%bash
source "$HOME/.sdkman/bin/sdkman-init.sh"
cd $REPO_ROOT
python3.8 tpcdi.py --output-directory $GEN_OUTPUT_DIR --stage $DATA_BUCKET

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/01/04 13:06:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/04 13:06:53 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: gs://tbd-2023z-304098-data/tpc-di/Date.txt.
java.lang.RuntimeException: java.lang.ClassNotFoundException: Class com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:36

╭───────────────────── Traceback (most recent call last) ──────────────────────╮
│ /home/jupyter/git/tbd-tpc-di/tpcdi.py:139 in process_files                   │
│                                                                              │
│   136 │   │   │   StructField("FISCAL_QTR_DESC", StringType(), False),       │
│   137 │   │   │   StructField("HOLIDAY_FLAG", BooleanType(), False),         │
│   138 │   │   ])                                                             │
│ ❱ 139 │   │   load_csv(schema, con_file_name, 'date')                        │
│   140 │                                                                      │
│   141 │   con_file_name = 'DailyMarket.txt'                                  │
│   142 │   if file_name in ['all', con_file_name]:                            │
│                                                                              │
│ ╭───────────────────────────────── locals ─────────────────────────────────╮ │
│ │            batch = 1    

CalledProcessError: Command 'b'source "$HOME/.sdkman/bin/sdkman-init.sh"\ncd $REPO_ROOT\npython3.8 tpcdi.py --output-directory $GEN_OUTPUT_DIR --stage $DATA_BUCKET\n'' returned non-zero exit status 1.

## Run dbt ELT

In [39]:
%%bash
cd $REPO_ROOT
dbt run

bash: line 2: dbt: command not found


CalledProcessError: Command 'b'cd $REPO_ROOT\ndbt run\n'' returned non-zero exit status 127.

## Run dbt tests

In [ ]:
%%bash
cd $REPO_ROOT
dbt test

In [40]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("TBD-TPC-DI-setup") \
    .master("yarn") \
    .enableHiveSupport() \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/01/04 13:08:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/04 13:08:42 WARN DomainSocketFactory: The short-circuit local reads feature cannot be used because libhadoop cannot be loaded.
24/01/04 13:08:43 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [41]:
spark.sql("show databases").show()

24/01/04 13:09:41 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/01/04 13:09:41 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/01/04 13:09:52 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
24/01/04 13:09:52 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@172.17.0.2
24/01/04 13:09:52 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException
+---------+
|namespace|
+---------+
|  default|
+---------+



In [42]:
spark.sql("use demp_gold")

24/01/04 13:09:56 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
24/01/04 13:09:56 WARN ObjectStore: Failed to get database demp_gold, returning NoSuchObjectException


AnalysisException: Database 'demp_gold' not found

In [43]:
spark.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [44]:
spark.stop()